# Hedging U.S. Recessions<br>
### Authors: Tejas Appana, Drew Carranti, Muhammad Elfayoumi<br>


In [ ]:
# Default Imports
import os
import sys
import time
import warnings
from loguru import logger

# Data Manipulation
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt

# Data Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns

# Module Imports
from classes.asset_portfolio import AssetPortfolio
from classes.objects import import_data
from classes.objects import color_palette, rf

# Presets
%matplotlib inline
warnings.filterwarnings("ignore")

# From CLEAN_DATA folder, import all data
data_dict = import_data("CLEAN_DATA", ignore_files=["README.md"])

## Introduction
The possibility of a U.S. recession in the next 12 months, as predicted by the Federal Reserve Bank of New York, has raised concerns about the stability of the American economy. Using credit spreads, the bank has determined a 57.77% chance of a recession, which suggests that the economy may experience a significant pullback. This situation highlights the need for investors to be mindful of alternative investment options to protect their wealth and minimize risks.

Moreover, traditional safe havens, such as the U.S. dollar, may not be immune to inflationary pressures and its weakening against other currencies. These factors have resulted in the critical need for investors to explore other investment vehicles that can hedge against U.S. market volatility, particularly during recessionary periods. Therefore, in this paper, we will examine the ability of alternative asset classes to provide a hedge against market volatility and discuss how investors should use them. We will also provide insights on the best practices for investors to navigate the current market conditions and achieve their investment goals.


In [ ]:
# Define the ticker symbol and get the data
vix_df = yf.download("^VIX", period="max", progress=False)
spy_df = yf.download("^GSPC", period="max", progress=False)

joined_df = vix_df.join(spy_df, how="inner", lsuffix=".V", rsuffix=".S", on="Date")
joined_df = joined_df[["Adj Close.V", "Adj Close.S"]]
joined_df["Adj Close.S"] = joined_df["Adj Close.S"] / joined_df["Adj Close.S"].iloc[0]
joined_df["Adj Close.V"] = joined_df["Adj Close.V"] / joined_df["Adj Close.V"].iloc[0]


# Blocks of recession periods and bullish periods highlighted in the graph
recession_periods = [
    pd.date_range(start='1990-07-01', end='1991-03-01'),
    pd.date_range(start='2001-03-01', end='2001-11-01'),
    pd.date_range(start='2007-12-01', end='2009-06-01'),
    pd.date_range(start='2020-02-01', end='2020-08-07')
]

bullish_periods = [
    pd.date_range(start = '1991-03-01', end = '2000-03-01'),
    pd.date_range(start = '2002-10-01', end = '2007-10-01'),
    pd.date_range(start = '2009-06-01', end = '2020-02-01'),
    pd.date_range(start = '2020-08-07', end = '2021-12-31')
]

# Create an interactive plot using plotly express
fig = px.line(title="VIX & SPY during Recessionary (red) and Bullish (grey) Periods")
fig.add_scatter(x=joined_df.index, y=joined_df["Adj Close.V"], name="VIX", line_color=color_palette["stevens red"])
fig.add_scatter(x=joined_df.index, y=joined_df["Adj Close.S"], name="SPY", line_color=color_palette["black"])
fig.update_layout(title_x=0.55, xaxis_title='Date', yaxis_title='Normalized Price', plot_bgcolor='white', 
                  paper_bgcolor='white', font_color=color_palette["black"], width=800, height=500, xaxis_gridcolor=color_palette["light grey"], yaxis_gridcolor=color_palette["light grey"])
fig.update_xaxes(showline=True, linewidth=2, linecolor=color_palette["dark grey"])
fig.update_yaxes(showline=True, linewidth=2, linecolor=color_palette["dark grey"])
fig.update_traces(showlegend=True)

# Add recession periods to the plot
for period in recession_periods:
    fig.add_vrect(x0=period[0], x1=period[-1], fillcolor=color_palette["stevens red"], opacity=0.15, line_width=0)

# Add bullish periods to the plot
for period in bullish_periods:
    fig.add_vrect(x0=period[0], x1=period[-1], fillcolor=color_palette["stevens grey"], opacity=0.15, line_width=0)


# Center the VIX time series data with respect to the y-axis
fig.update_yaxes(automargin=True)

# Show the plot
fig.show()

### Alternative Asset Classes & Indices
In our assessment of alternative asset classes to hedge against U.S. market volatility, we will consider a diverse set of options. The Nasdaq Composite (QQQ) and Dow Jones Industrial Average (DIA) are equity index funds that represent a broad spectrum of the U.S. stock market. We will also evaluate the iShares 1-3 Year Treasury Bond ETF (SHY), iShares 7-10 Year Treasury Bond ETF (IEF), and iShares 20+ Year Treasury Bond ETF (TLT), which represent the U.S. government bonds of various maturities.

Furthermore, we will analyze the Vanguard Real Estate Index Fund (VNQ), which invests in U.S. real estate investment trusts (REITs). We will also consider the ICE Futures US Dollar Index (DX-Y.NYB), which tracks the value of the U.S. dollar relative to a basket of other major currencies. In addition, we will evaluate the S&P 500 Consumer Staples Sector (^SP500-30), which includes companies that produce essential consumer goods.

We will also investigate a mix of traditional and non-traditional assets, such as gold (GC=F), Bitcoin (BTC-USD), Ethereum (ETH-USD), Brent Crude Oil (BZ=F), Corn Futures (ZC=F), CBOE Volatility Index (^VIX), CME Eurodollar Futures (GE=F), and Chicago SRW Wheat Futures (ZW=F). These assets have varying degrees of correlation with the U.S. stock market, and we will evaluate their effectiveness in mitigating risks during market downturns.


### Sourcing Data & Preprocessing
To evaluate the effectiveness of the selected asset classes in hedging against U.S. market volatility, we sourced daily data from Yahoo Finance API for all the assets and indices under consideration, including their respective benchmarks for comparison. We obtained the Adjusted Close Prices and Volume for all the assets, indices, and benchmarks to enable meaningful analysis.


Before performing any analysis, we preprocessed the data by interpolating null values and calculated daily returns using percent change. We then transformed the daily returns into log returns for normalization purposes. This preprocessing step ensures that the data is ready for statistical analysis and minimizes errors in our results.

Additionally, we will compare the performance of each asset class with the market returns, represented by the S&P 500, and market volatility, represented by the CBOE Volatility Index (VIX). By comparing our results with these benchmarks, we can determine whether the selected assets provide better risk-adjusted returns during market downturns. Overall, this analysis will help investors identify the most effective asset classes to minimize risks and maximize returns during periods of market volatility.


### Methodology
The goal of this study is to test the ability of various asset classes and indices to hedge against the overall U.S. market. To achieve this objective, we designed two asset portfolios, each combining an asset class or index with the S&P 500, with an equal weighting of each component.

To determine which assets are most effective as hedges, we analyzed the statistics of both individual assets and indices, as well as the statistics of each of the respective portfolios. This analysis will allow us to identify the assets that are most likely to perform well in a hedging context.

To assess the overall performance of these portfolios against market behavior, we will evaluate their performance during both recessionary and bullish periods. By doing so, we will gain insight into the extent of each asset's ability to hedge against the broader market.

The results of this study will contribute to the body of knowledge on asset allocation and risk management strategies. Specifically, it will shed light on the efficacy of different asset classes and indices as hedges against market risk, which can inform investment decisions for both individual and institutional investors. The methodology employed in this study, which combines statistical analysis with portfolio design, can also serve as a useful framework for future research on this topic.


## Portfolio Analyses

In [ ]:
recession_portfolios = {}
bullish_portfolios = {}
full_portfolios = {}

# Replace all values in data_dict with AssetPortfolio objects
for k in data_dict.keys():
    # logger.info(f"Creating Portfolios for {k}")
    full_portfolios[k] = AssetPortfolio(k)
    recession_portfolios[k] = AssetPortfolio(k, "recession")
    bullish_portfolios[k] = AssetPortfolio(k, "bullish")

### Metrics

In [ ]:
# Import csvs from OUTPUT_FILES folder
full_metrics = pd.read_csv("OUTPUT_FILES/metrics.csv", index_col="Unnamed: 0")
recession_metrics = pd.read_csv("OUTPUT_FILES/metrics_recession.csv", index_col="Unnamed: 0")
bullish_metrics = pd.read_csv("OUTPUT_FILES/metrics_bullish.csv", index_col="Unnamed: 0")

In [ ]:
full_metrics

In [ ]:
recession_metrics

In [ ]:
bullish_metrics

### Correlation Analysis

In [ ]:
df = pd.DataFrame({
    "Recession": recession_metrics["Correlation"],
    "Bullish": bullish_metrics["Correlation"],
    "Aggregate": full_metrics["Correlation"]
    }, index=full_metrics.index
)

# create the plot
# Set the bar width
bar_width = 0.25

# Set the positions of the bars on the x-axis
pos_1 = np.arange(len(df))
pos_2 = [x + bar_width for x in pos_1]
pos_3 = [x + bar_width for x in pos_2]

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(pos_1, df["Recession"], color=color_palette['stevens red'], width=bar_width, label='Recession')
rects2 = ax.bar(pos_2, df["Bullish"], color=color_palette["stevens grey"], width=bar_width, label='Bullish')
rects3 = ax.bar(pos_3, df["Aggregate"], color=color_palette["black"], width=bar_width, label='Aggregate')

# Add x-axis labels and tick marks
ax.set_xticks([p + 1.5 * bar_width for p in pos_1])
ax.set_xticklabels(df.index, rotation=45, fontsize=10)

# Add legend and title
ax.legend()
ax.set_title('Correlation of Assets to Market', fontsize=14, fontweight='bold')

# set the axis label
ax.set_xlabel("Assets")
ax.set_ylabel("Correlation")

# create gridlines
ax.grid(color=color_palette["dark grey"], linestyle='-', linewidth=0.25)

# change the color of the graph
# ax.set_facecolor(color_palette["light grey"])

# change the color of the x-axis and y-axis
for spine in ax.spines.values():
    spine.set_edgecolor(color_palette["dark grey"])

# add annotations to each bar
# for rect in rects1:
#     height = rect.get_height()
#     if height < 0:
#         ax.text(rect.get_x() + rect.get_width() / 2, height, f"{height:.2f}", ha='center', va='top', fontsize=6)
#     else:
#         ax.text(rect.get_x() + rect.get_width() / 2, height, f"{height:.2f}", ha='center', va='bottom', fontsize=6)
    
# for rect in rects2:
#     height = rect.get_height()
#     if height < 0:
#         ax.text(rect.get_x() + rect.get_width() / 2, height, f"{height:.2f}", ha='center', va='top', fontsize=6)
#     else:
#         ax.text(rect.get_x() + rect.get_width() / 2, height, f"{height:.2f}", ha='center', va='bottom', fontsize=6)

for rect in rects3:
    height = rect.get_height()
    if height < 0:
        ax.text(rect.get_x() + rect.get_width() / 2, height - 0.03, f"{height:.2f}", ha='center', va='top', fontsize=8)
    else:
        ax.text(rect.get_x() + rect.get_width() / 2, height + 0.03, f"{height:.2f}", ha='center', va='bottom', fontsize=8)

# show the plot
plt.show()

Our analysis revealed that some of the popular indices such as the Nasdaq, Dow Jones, Consumer Staples, HealthCare, and Real Estate exhibit high correlations with the overall market and may not be effective hedges during periods of market volatility. In contrast, alternative asset classes such as Bitcoin, Ethereum, Corn, Euro, Gold, Wheat, and Oil show lower correlations with the market, suggesting some potential for hedging.

However, it is important to note that ideally, we would prefer assets that exhibit negative correlations with the overall market during market downturns. Among the assets we evaluated, only T-bills, USD, and the VIX showed negative correlations with the overall market. This is an encouraging sign since these assets are typically considered safe-haven assets during market downturns, and our analysis confirms their ability to hedge against market volatility.

### Risk Analysis

In [ ]:
df = recession_metrics[["Market Volatility", "Asset Volatility", "Portfolio Volatility"]]
# Set the positions of the bars on the x-axis
pos_1 = np.arange(len(df))
pos_2 = [x + bar_width for x in pos_1]
pos_3 = [x + bar_width for x in pos_2]

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(pos_1, df["Market Volatility"], color=color_palette['stevens grey'], width=bar_width, label='Market')
rects2 = ax.bar(pos_2, df["Asset Volatility"], color=color_palette["stevens red"], width=bar_width, label='Asset')
rects3 = ax.bar(pos_3, df["Portfolio Volatility"], color=color_palette["black"], width=bar_width, label='Portfolio')

# Add x-axis labels and tick marks
ax.set_xticks([p + 1.5 * bar_width for p in pos_1])
ax.set_xticklabels(df.index, rotation=45, fontsize=10)

# Add legend and title
ax.legend()
ax.set_title('Volatilities of Assets and Portfolios (Recession)', fontsize=14, fontweight='bold')

# change the color of the graph
# ax.set_facecolor(color_palette["light grey"])

# change the color of the x-axis and y-axis
for spine in ax.spines.values():
    spine.set_edgecolor(color_palette["dark grey"])

# add annotations to each bar
for rect in rects3:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2 + 0.175, height + 0.02, f"{height:.2f}", ha='center', va='bottom', fontsize=8)


# set the axis label
ax.set_xlabel("Assets")
ax.set_ylabel("Volatility")

# create gridlines
ax.grid(color=color_palette["dark grey"], linestyle='-', linewidth=0.25)

# show the plot
plt.show()

Our analysis evaluated the volatility of portfolios containing various assets during a recessionary period. We found that portfolios containing Bitcoin, Ethereum, and the real estate ETF had the highest volatility, indicating greater risk exposure during a market downturn. On the other hand, portfolios containing the Euro, USD, and the three T-Bills demonstrated the lowest volatility during a recessionary period. These assets are considered safe-haven assets that typically perform well during market downturns, and our analysis confirms their effectiveness in reducing risk exposure during times of economic stress.

In [ ]:
df = bullish_metrics[["Market Volatility", "Asset Volatility", "Portfolio Volatility"]]
# Set the positions of the bars on the x-axis
pos_1 = np.arange(len(df))
pos_2 = [x + bar_width for x in pos_1]
pos_3 = [x + bar_width for x in pos_2]

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(pos_1, df["Market Volatility"], color=color_palette['stevens grey'], width=bar_width, label='Market')
rects2 = ax.bar(pos_2, df["Asset Volatility"], color=color_palette["stevens red"], width=bar_width, label='Asset')
rects3 = ax.bar(pos_3, df["Portfolio Volatility"], color=color_palette["black"], width=bar_width, label='Portfolio')

# Add x-axis labels and tick marks
ax.set_xticks([p + 1.5 * bar_width for p in pos_1])
ax.set_xticklabels(df.index, rotation=45, fontsize=10)

# Add legend and title
ax.legend()
ax.set_title('Volatilities of Assets and Portfolios (Bullish)', fontsize=14, fontweight='bold')

# change the color of the graph
# ax.set_facecolor(color_palette["light grey"])

# change the color of the x-axis and y-axis
for spine in ax.spines.values():
    spine.set_edgecolor(color_palette["dark grey"])

# add annotations to each bar
for rect in rects3:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2 + 0.175, height + 0.02, f"{height:.2f}", ha='center', va='bottom', fontsize=8)


# set the axis label
ax.set_xlabel("Assets")
ax.set_ylabel("Volatility")

# create gridlines
ax.grid(color=color_palette["dark grey"], linestyle='-', linewidth=0.25)

# show the plot
plt.show()

Our analysis also included the evaluation of the portfolios during a bullish period. Our results show that the portfolios containing Bitcoin, Ethereum, and the VIX had the highest volatility during this period. This could be attributed to the fact that these assets have a higher risk profile and are often viewed as more speculative investments. In contrast, the portfolios containing EURO, USD, and the 3 T-Bills had the lowest volatility during the same period. These portfolios exhibited a lower risk profile and are considered more conservative investments, which could explain their lower volatility.

In [ ]:
df = full_metrics[["Market Volatility", "Asset Volatility", "Portfolio Volatility"]]
# Set the positions of the bars on the x-axis
pos_1 = np.arange(len(df))
pos_2 = [x + bar_width for x in pos_1]
pos_3 = [x + bar_width for x in pos_2]

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(pos_1, df["Market Volatility"], color=color_palette['stevens grey'], width=bar_width, label='Market')
rects2 = ax.bar(pos_2, df["Asset Volatility"], color=color_palette["stevens red"], width=bar_width, label='Asset')
rects3 = ax.bar(pos_3, df["Portfolio Volatility"], color=color_palette["black"], width=bar_width, label='Portfolio')

# Add x-axis labels and tick marks
ax.set_xticks([p + 1.5 * bar_width for p in pos_1])
ax.set_xticklabels(df.index, rotation=45, fontsize=10)

# Add legend and title
ax.legend()
ax.set_title('Volatilities of Assets and Portfolios (Aggregate)', fontsize=14, fontweight='bold')

# change the color of the graph
# ax.set_facecolor(color_palette["light grey"])

# change the color of the x-axis and y-axis
for spine in ax.spines.values():
    spine.set_edgecolor(color_palette["dark grey"])

# add annotations to each bar
for rect in rects3:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2 + 0.175, height + 0.02, f"{height:.2f}", ha='center', va='bottom', fontsize=8)


# set the axis label
ax.set_xlabel("Assets")
ax.set_ylabel("Volatility")

# create gridlines
ax.grid(color=color_palette["dark grey"], linestyle='-', linewidth=0.25)

# show the plot
plt.show()

Our analysis of the different portfolios revealed varying levels of volatility based on the inclusion of different assets. Portfolios that included Bitcoin, Ethereum, and the VIX demonstrated the highest levels of volatility in aggregate. In contrast, portfolios that included the EURO, USD, Gold, and the three T-Bills exhibited the lowest levels of volatility in aggregate. It is important to note that volatility, while a measure of risk, is not necessarily indicative of returns. Higher volatility portfolios may offer higher returns in exchange for the increased risk, while lower volatility portfolios may offer more stable but lower returns as we will explore in the next section.

### Excess Return Analysis

In [ ]:
# Set the positions of the bars on the x-axis
pos_1 = np.arange(len(df))
pos_2 = [x + bar_width for x in pos_1]
pos_3 = [x + bar_width for x in pos_2]

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(pos_1, recession_metrics["Excess Return"], color=color_palette['stevens red'], width=bar_width, label='Recessionary')
rects2 = ax.bar(pos_2, bullish_metrics["Excess Return"], color=color_palette["stevens grey"], width=bar_width, label='Bullish')
rects3 = ax.bar(pos_3, full_metrics["Excess Return"], color=color_palette["black"], width=bar_width, label='Aggregate')

# Add x-axis labels and tick marks
ax.set_xticks([p + 1.5 * bar_width for p in pos_1])
ax.set_xticklabels(df.index, rotation=45, fontsize=10)

# Add legend and title
ax.legend()
ax.set_title('Excess Return Comparison (vs Risk-Free Rate)', fontsize=14, fontweight='bold')

# change the color of the graph
# ax.set_facecolor(color_palette["light grey"])

# change the color of the x-axis and y-axis
for spine in ax.spines.values():
    spine.set_edgecolor(color_palette["dark grey"])

# add annotations to each bar
for rect in rects3:
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2 + 0.175, height + 0.02, f"{height:.2f}", ha='center', va='bottom', fontsize=8)


# set the axis label
ax.set_xlabel("Assets")
ax.set_ylabel("Percent Return")

# create gridlines
ax.grid(color=color_palette["dark grey"], linestyle='-', linewidth=0.25)

# show the plot
plt.show()

In [ ]:
# 1-D matrix of T-Testing of mean returns of portfolios
# Create empty matrix
matrix = pd.DataFrame(columns=["Recession", "Bullish", "Aggregate"], index=full_metrics.index)
# Loop through all combinations of 2 portfolios
for k in full_portfolios.keys():
    matrix.loc[k, "Recession"] = recession_portfolios[k].test_returns()
    matrix.loc[k, "Bullish"] = bullish_portfolios[k].test_returns()
    matrix.loc[k, "Aggregate"] = full_portfolios[k].test_returns()


mat = matrix.astype(bool)

# Create the heatmap
sns.heatmap(mat, cmap=sns.color_palette([color_palette["stevens red"], color_palette["stevens grey"]]), cbar=False, annot=True)

# Add title and labels
plt.title('Are Portfolio Returns Statistically Greater than Risk-Free Rate?')

# Show the plot
plt.show()



Our analysis also evaluated the aggregate excess returns of the portfolios. We found that the Bitcoin portfolio had the highest excess return, averaging about 0.2% higher than the current risk-free rate. However, when we examine performance during recessionary periods, we see that the Ethereum and Bitcoin portfolios exhibit particularly high excess returns at 0.9% and 0.6% respectively, suggesting a strong ability to hedge against market downturns.

In contrast, almost all other portfolios underperformed the risk-free rate during recessionary periods, making them poor choices for hedging against market volatility. These findings underscore the importance of careful portfolio selection, especially during periods of economic uncertainty. While some assets may offer attractive returns under normal market conditions, their effectiveness as hedges during downturns must also be considered.


### Sharpe Ratio Analysis

In [ ]:
# Set the positions of the bars on the x-axis
pos_1 = np.arange(len(df))
pos_2 = [x + bar_width for x in pos_1]
pos_3 = [x + bar_width for x in pos_2]

# Create the bar chart
fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(pos_1, recession_metrics["Sharpe Ratio"], color=color_palette['stevens red'], width=bar_width, label='Recessionary')
rects2 = ax.bar(pos_2, bullish_metrics["Sharpe Ratio"], color=color_palette["stevens grey"], width=bar_width, label='Bullish')
rects3 = ax.bar(pos_3, full_metrics["Sharpe Ratio"], color=color_palette["black"], width=bar_width, label='Aggregate')

# Add x-axis labels and tick marks
ax.set_xticks([p + 1.5 * bar_width for p in pos_1])
ax.set_xticklabels(df.index, rotation=45, fontsize=10)

# Add legend and title
ax.legend()
ax.set_title('Sharpe Ratio Comparison', fontsize=14, fontweight='bold')

# change the color of the graph
# ax.set_facecolor(color_palette["light grey"])

# change the color of the x-axis and y-axis
for spine in ax.spines.values():
    spine.set_edgecolor(color_palette["dark grey"])

# add annotations to each bar
for rect in rects3:
    height = rect.get_height()
    if height < 0:
        ax.text(rect.get_x() + rect.get_width() / 2, height - 0.02, f"{height:.2f}", ha='center', va='top', fontsize=8)
    else:
        ax.text(rect.get_x() + rect.get_width() / 2, height + 0.02, f"{height:.2f}", ha='center', va='bottom', fontsize=8)


# set the axis label
ax.set_xlabel("Assets")
ax.set_ylabel("Percent Return")

# create gridlines
ax.grid(color=color_palette["dark grey"], linestyle='-', linewidth=0.25)

# show the plot
plt.show()

Our analysis reveals that Ethereum and Bitcoin offer the best return relative to risk during recessionary periods, making them a potentially attractive investment for those looking to hedge against market volatility during these times. However, outside of recessionary periods, indices, long-term treasuries, and cryptocurrencies provide the best risk-return tradeoff. Therefore, investors need to consider their investment objectives and market conditions when selecting their investment portfolios.

When it comes to constructing an investment portfolio, our analysis shows that a combination of either Bitcoin, long-term T-bills, or Gold produce the best risk-return tradeoff in an aggregate context. These assets provide a well-diversified portfolio that can potentially offer stable returns while managing risk effectively. On the other hand, ETH, USD, and the Euro produce the worst portfolios in terms of risk-return tradeoff in aggregate.


## Conclusions

Our analysis yielded several key findings that provide valuable insights into effective hedging strategies for the U.S. market. In terms of correlation analysis, we found that long-term treasury bills and the VIX were the most effective hedges against market risk. This suggests that investors seeking to mitigate exposure to market fluctuations should consider allocating a portion of their portfolio to these assets.

When it comes to risk management, our analysis revealed that the Euro and T-bills produced the best portfolios in terms of minimizing risk. This highlights the importance of diversifying across asset classes and regions in order to achieve a well-balanced and resilient portfolio.

Our excess return analysis identified Bitcoin and Ethereum as potentially strong hedges in terms of relative excess returns. While these assets carry higher risk, they also offer the potential for greater returns, making them an attractive option for investors willing to take on additional risk.

Further analysis of Sharpe ratios revealed that Bitcoin and Ethereum produced the best portfolios during recessionary periods, while indices and T-bills were optimal in bullish markets. In aggregate, our analysis suggests that Bitcoin, Gold, and T-bills produced the best portfolios. 

It's important to recognize that past performance is not necessarily indicative of future results. While cryptocurrencies performed well in certain areas, such as recessions, they also experienced bullish runs at those times which may not be the case for future recessions. Therefore, investors should exercise caution when considering these assets as part of their investment strategy.

Overall, these conclusions underscore the importance of carefully selecting assets for inclusion in a well-diversified portfolio, based on a thorough analysis of their risk and return characteristics. By doing so, investors can better position themselves to weather market volatility and achieve their long-term investment goals.


Our analysis also evaluated the aggregate excess returns of the portfolios. We found that the Bitcoin portfolio had the highest excess return, averaging about 0.2% higher than the current risk-free rate. However, when we examine performance during recessionary periods, we see that the Ethereum and Bitcoin portfolios exhibit particularly high excess returns at 0.9% and 0.6% respectively, suggesting a strong ability to hedge against market downturns.

In contrast, almost all other portfolios underperformed the risk-free rate during recessionary periods, making them poor choices for hedging against market volatility. These findings underscore the importance of careful portfolio selection, especially during periods of economic uncertainty. While some assets may offer attractive returns under normal market conditions, their effectiveness as hedges during downturns must also be considered.
